In [106]:
if False:
    !pip install wget
    !pip install systemml

In [107]:
import os
import wget
import pandas as pd

#repository path URL
PATH='https://raw.githubusercontent.com/villegasjuanf/heartdigs/master'

#get file from repository
def get_file(path,filename):
    if filename not in os.listdir():
        try:
            wget.download(os.path.join(path,filename))
        except:
            return None
            print('archivo %s inexistente'%filename)
        else:
            return filename
            print('file %s downloaded'%filename)

#download table with filenames and labels
currdir=os.curdir
if 'set_a.csv' not in os.listdir(currdir):
    get_file(PATH,'set_a.csv')

audio_files_df=pd.read_csv('set_a.csv')
audio_files_df=audio_files_df.loc[audio_files_df.label.isin(['normal','murmur','extrahls']),['fname','label']]
audio_files_df['fname']=audio_files_df.fname.apply(lambda x: x[6:len(x)]).tolist()

print('\nimported categories\n============================')
print(audio_files_df.groupby('label').count())
print('\nimported audio files\n============================')
print(audio_files_df)


imported categories
          fname
label          
extrahls     19
murmur       34
normal       31

imported audio files
                          fname     label
40   extrahls__201101070953.wav  extrahls
41   extrahls__201101091153.wav  extrahls
42   extrahls__201101152255.wav  extrahls
43   extrahls__201101160804.wav  extrahls
44   extrahls__201101160808.wav  extrahls
45   extrahls__201101161027.wav  extrahls
46   extrahls__201101241423.wav  extrahls
47   extrahls__201101241433.wav  extrahls
48   extrahls__201102070251.wav  extrahls
49   extrahls__201102071835.wav  extrahls
50   extrahls__201102241217.wav  extrahls
51   extrahls__201103150114.wav  extrahls
52   extrahls__201103170134.wav  extrahls
53   extrahls__201103182227.wav  extrahls
54   extrahls__201103200218.wav  extrahls
55   extrahls__201104021355.wav  extrahls
56   extrahls__201104140118.wav  extrahls
57   extrahls__201104270458.wav  extrahls
58   extrahls__201104270459.wav  extrahls
59     murmur__201101051104.wav    mu

In [108]:
LOAD_MODEL=False # use TRUE to load_models the first time
selected_file='murmur__201108222248.wav' #if selected_file is None a random file is extracted from repository

from scipy.io import wavfile
import numpy as np

if selected_file is None:
    selected_file=audio_files_df.fname.sample().iloc[0]
label=audio_files_df.loc[audio_files_df.fname==selected_file,'label'].iloc[0]

print('selected file: %s '%selected_file)
print('label: %s\n'%label)

if selected_file not in os.listdir(currdir):
    get_file(PATH,selected_file) #audio file is downloaded from repository

f_,data_=wavfile.read(selected_file)
selected_data=np.array(data_) #audio file data

print('archivo | dimension | frec muestreo')
print(selected_file,data_.shape[0],f_)

selected file: murmur__201108222248.wav 
label: murmur

archivo | dimension | frec muestreo
('murmur__201108222248.wav', 349958, 44100)


In [109]:
from scipy.signal import decimate

DOWNSAMPLE=60 # resampling to fs/DOWNSAMPLE
N_DATA=4000 # crop data to first N_DATA
CREATE_M=True #create a new DFT matrix M (use TRUE once if you change N_DATA or DOWNSAMPLE)

#resampling and padding to fit each data serie to N_DATA lenght

fs=f_/DOWNSAMPLE
data_dec= decimate(selected_data,q=DOWNSAMPLE,ftype='fir')[0:N_DATA] #resample data to a nyquist frequency of 367.5Hz
if len(data_dec)<N_DATA:
    data_dec=np.append(data_dec,[0]*(N_DATA-len(data_dec)))

print("downsampled data\n========================================\ndata points: %d  sample frequency: %0.1f"%(len(data_dec),fs))

downsampled data
data points: 4000  sample frequency: 735.0


In [110]:
from systemml import MLContext, dml
ml = MLContext(spark)

#script to create and save DFT matrix using cosine transform Ma=cos(M)
dml_matrix_a = '''
PI = 3.141592654

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = cos((n %*% t(k))*(2*PI/N))
print(nrow(M)+" "+ncol(M))
write(M,'Ma.mm',format='mm')
'''
# DFT cosine signal transformation
dml_dft='''
Ma=read('Ma.mm')
Xa = Ma %*% signal
DFT= Xa
'''

#matrix creation script execution
if(CREATE_M):
    prog_a = dml(dml_matrix_a).input('N', N_DATA)
    ml.execute(prog_a)

In [111]:
#funcion de calculo de la DFT
def dft_systemml(signal):
    prog = dml(dml_dft).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toNumPy() 
    )

#calculo de la DFT de todos los archivos de audio
data_=np.array(data_dec).reshape([N_DATA,1])
print("DFT data\n============================================\n%s: (%d , %d)\n"%(selected_file,data_.shape[0],data_.shape[1]))
data_dft=dft_systemml(data_)[0:N_DATA]

DFT data
murmur__201108222248.wav: (4000 , 1)



In [112]:
from bokeh.io import output_notebook
from bokeh.layouts import row,column
from bokeh.plotting import figure,show,ColumnDataSource
from bokeh.palettes import viridis
output_notebook()

n_skip=10

#time domain plot
xp=np.arange(0,float(len(data_dec))/fs,1./fs)
p=figure(title='auscultation audio wave')
p.line(x=xp,y=data_dec)

#frequency domain plot
xq=np.arange(0,N_DATA//2-1,n_skip)*fs/N_DATA
yq=data_dft[:N_DATA//2:n_skip].flatten()
q=figure(title='auscultation frequency spectra')
q.vbar(x=xq,bottom=0,width=2,top=20*np.log(np.power(yq,2)))

show(row([p,q]))

Loading BokehJS ...

In [113]:
import h5py
from keras.models import Sequential,model_from_json
from keras.layers import Conv1D,MaxPooling1D, Dropout,Dense,Activation,Flatten
from keras.utils import to_categorical
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder,PCA
from pyspark.ml import Pipeline,PipelineModel

In [114]:
if LOAD_MODEL:
    model_name='model(bs1 f199 ks16 h23 ps18) acc0.95_'

    #model loading
    with open(model_name+'.json','r') as f:
        model_json=f.read()
    model=model_from_json(model_json)
    model.load_weights(model_name+'.h5')
    xpipe=PipelineModel.load(model_name+'.xpip')
    ypipe=PipelineModel.load(model_name+'.ypip')

In [115]:
data_dft=np.log(np.power(data_dft[0:N_DATA//2],2)) #frequency data is transformed to dB
data_dft=data_dft/max(data_dft) #data is normalized

#PySpark DataFrame creation
input_data=pd.DataFrame(data_dft).transpose()
input_data['label']=label
input_data['filename']=selected_file
input_data['fs']=float(fs)/2
input_data['k']=N_DATA//2

spark=SparkSession.builder.getOrCreate()
input_data=spark.createDataFrame(input_data)
input_data.toPandas()

,0,1,2,3,4,5,6,7,8,9,...,1994,1995,1996,1997,1998,1999,label,filename,fs,k
0,0.612872,0.655196,0.71594,0.530863,0.668412,0.607797,0.67328,0.566275,0.685017,0.607891,...,0.494257,0.461986,0.517411,0.481866,0.573744,0.500885,murmur,murmur__201108222248.wav,367.5,2000


In [116]:
# X preprocessing pipeline: VectorAssembler + PCA(140 comp)
x_data=xpipe.transform(input_data).select('freq_channels').collect()
x_data=np.array(x_data[0]['freq_channels']).reshape([1,140,1])
print(xpipe.stages)

# Y preprocessing pipeline: StringIndexer + OneHotEncoder
y_data=ypipe.transform(input_data).select('out').collect()
y_data=np.array(y_data[0]['out']).reshape((2,))
print(ypipe.stages)

#1-D Convolutional Neural Network
result=model.predict(x_data).reshape((2,))
model.summary()

def print_result(x):
    print('normal: %1.3f murmur: %1.3f extrahls: %1.3f'%(x[0],x[1],np.sqrt((1-x[0])*(1-x[1]))))

[VectorAssembler_44218993468f4a6febfe, PCA_447aae980a6f87ab56d5]
[StringIndexer_477c9d69f24c048d347f, OneHotEncoder_442e84beafa3acd835bf]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_807 (Conv1D)          (None, 140, 199)          3383      
_________________________________________________________________
max_pooling1d_807 (MaxPoolin (None, 7, 199)            0         
_________________________________________________________________
dropout_807 (Dropout)        (None, 7, 199)            0         
_________________________________________________________________
flatten_764 (Flatten)        (None, 1393)              0         
_________________________________________________________________
dense_1529 (Dense)           (None, 23)                32062     
_________________________________________________________________
activation_1529 (Activation) (None, 23)                0         
____

In [117]:
print('results for file: %s\n'%selected_file)

print('true result\n================================================')
print_result(y_data)
print('\nresult\n================================================')
print_result(result)

results for file: murmur__201108222248.wav

true result
normal: 0.000 murmur: 1.000 extrahls: 0.000

result
normal: 0.382 murmur: 0.618 extrahls: 0.486
